# **KG-LLM**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!huggingface-cli login

In [ ]:
!pip install bitsandbytes
!pip install --upgrade pyarrow
!pip install --upgrade pandas datasets
!pip install sentencepiece
!pip install sentencepiece --force-reinstall
!pip install -U scikit-learn
!pip install torch
!pip install accelerate
!pip install datasets==2.13.1
!pip install transformers
!pip install peft
!pip install trl
!pip install scipy

In [ ]:
import pandas as pd
import json
from collections import deque
import csv
import random
from sklearn.model_selection import train_test_split
import os

In [ ]:
import argparse
import bitsandbytes as bnb
from datasets import load_dataset
from functools import partial
import os
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, AutoPeftModelForCausalLM
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, set_seed, Trainer, TrainingArguments, BitsAndBytesConfig, \
    DataCollatorForLanguageModeling, Trainer, TrainingArguments
from datasets import load_dataset

# *KG-LLM(CoT)*

**Preprocess data**

In [ ]:

input_file = open("train2id.txt", "r")
output_file = open("output.txt", "w")

# total number of lines
number = int(input_file.readline())

nodes = set()

graph = {}

for i in range(number):
    content = input_file.readline()
    node1, node2, relation = content.strip().split()

    nodes.add(node1)
    # nodes.add(node2)

    relation = int(relation)

    # Check if the first node already exists in the dictionary
    if node1 not in graph:
        # If not, create a new dictionary for the node
        graph[node1] = {}
    # Add the neighboring node and the relationship to the dictionary for node1
    graph[node1][node2] = relation


node_list = list(nodes)
node_list2 = list(nodes)

In [ ]:

os.environ["CUDA_VISIBLE_DEVICES"] = "1"

relation2id = {}

with open("relation2id.txt", "r") as file:
    relations = int(file.readline())
    for line in file:
        relation, relation_id = line.strip().split("\t")
        relation2id[int(relation_id)] = relation

unique_rows = set()

# change size if you want, but larger the size, smaller the data
size = 30
# how many positive and negative data set you want to train, the more dataset, the more time to train
total = 60000
fieldnames = ['Prompt', 'input_text', 'output_text']

instruction = 'Answer the following yes/no question by reasoning step-by-step. '

with open("train_data.csv", mode="w", newline='') as tra:
  with open("positive_data.csv", mode="w", newline='') as pos:
    with open("negative_data.csv",  mode="w", newline='') as neg:

        # Create a CSV writer object and write the headers to the file
        writer_pos = csv.DictWriter(pos, fieldnames=fieldnames)
        writer_pos.writeheader()

        writer_neg = csv.DictWriter(neg, fieldnames=fieldnames)
        writer_neg.writeheader()

        writer_tra = csv.DictWriter(tra, fieldnames=fieldnames)
        writer_tra.writeheader()


        def dfs(graph, size):
            pos_count = 0
            neg_count = 0
            times = 0
            term = True

            while times < total:
                visited = set()
                kg = []
                graph_size = random.randint(2, size)
                first_node = random.choice(node_list)
                visited.add(first_node)
                last_node = ""
                previous_node = first_node
                stack = [first_node]
                input_text = ""
                output_text = ""
                while len(visited) < graph_size:
                    if previous_node not in graph or set(graph[previous_node].keys()).issubset(visited):
                        node = random.choice(node_list)
                        while node in visited:
                            node = random.choice(node_list)
                        input_text += 'node_{} not connected with node_{}. '.format(previous_node, node)
                        output_text += 'node_{} not connected with node_{} means there is no relationship between node_{} and node_{}. '.format(previous_node, node, previous_node, node)
                        visited.add(node)
                        previous_node = node
                    else:
                        node = random.choice(list(graph[previous_node].keys()))
                        while node in visited:
                            node = random.choice(list(graph[previous_node].keys()))
                        relation = graph[previous_node][node]
                        r = relation2id[relation]
                        input_text += 'node_{} has relation_{} with node_{}. '.format(previous_node, relation, node)
                        output_text += 'node_{} has relation_{} with node_{}, means node_{} {} node_{}. '.format(previous_node, relation, node, previous_node, r, node)
                        visited.add(node)
                        previous_node = node
                    if len(visited) == graph_size:
                        last_node = previous_node

                # input_text += 'Answer the following yes/no question by reasoning step-by-step. Is the first node connnected with the last node?'
                was = len(unique_rows)
                unique_rows.add(input_text)
                if len(unique_rows) > was:
                  # if pos_count < int(total/2):
                    if first_node in graph and last_node in graph[first_node] and term:
                      # if pos_count < int(total/2):
                        final_relation = relation2id[graph[first_node][last_node]]
                        output_text += 'So node {} {} node {}. The answer is yes.'.format(first_node, final_relation, last_node)
                        prompt = 'Is node {} connnected with node {}?'.format(first_node, last_node)
                        comb = "###Instruction:\n" + instruction + prompt + "\n\n###Input:\n" + input_text + "\n\n###Response:\n" + output_text
                        writer_pos.writerow({'Prompt': comb, 'input_text': input_text + prompt, 'output_text': output_text})
                        writer_tra.writerow({'Prompt': comb, 'input_text': input_text + prompt, 'output_text': output_text})
                        pos_count += 1
                        term = False
                        times += 1
                    elif last_node in graph and first_node in graph[last_node] and term:
                      # if pos_count < int(total/2):
                        final_relation = relation2id[graph[last_node][first_node]]
                        output_text += 'So node {} {} node {}. The answer is yes.'.format(last_node, final_relation, first_node)
                        prompt = 'Is node {} connected with node {}?'.format(last_node, first_node)
                        comb = "###Instruction:\n" + instruction + prompt + "\n\n###Input:\n" + input_text + "\n\n###Response:\n" + output_text
                        writer_pos.writerow({'Prompt': comb, 'input_text': input_text + prompt, 'output_text': output_text})
                        writer_tra.writerow({'Prompt': comb, 'input_text': input_text + prompt, 'output_text': output_text})
                        pos_count += 1
                        term = False
                        times += 1
                  # elif neg_count < int(total//2):
                    elif not term:
                      # if neg_count < int(total/2):
                        output_text += 'So there is no connection between node {} and node {}. The answer is no.'.format(first_node, last_node)
                        prompt = 'Is node {} connected with node {}?'.format(first_node, last_node)
                        comb = "###Instruction:\n" + instruction + prompt + "\n\n###Input:\n" + input_text + "\n\n###Response:\n" + output_text
                        writer_neg.writerow({'Prompt': comb, 'input_text': input_text + prompt, 'output_text': output_text})
                        writer_tra.writerow({'Prompt': comb, 'input_text': input_text + prompt, 'output_text': output_text})
                        neg_count += 1
                        term = True
                        times += 1
                else:
                  continue


            print(pos_count)
            print(neg_count)

        dfs(graph, size)

30000
30000


In [ ]:
df1 = pd.read_csv('train_data.csv')
df1.sample(5)

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

positive_df = pd.read_csv('positive_data.csv')
negative_df = pd.read_csv('negative_data.csv')

train_pos, test_pos = train_test_split(positive_df, test_size=0.2, random_state=42)
train_neg, test_neg = train_test_split(negative_df, test_size=0.2, random_state=42)

train_df = pd.concat([train_pos, train_neg])
test_df = pd.concat([test_pos, test_neg])

train_df = train_df.sample(frac=1, random_state=42).reset_index(drop=True)
test_df = test_df.sample(frac=1, random_state=42).reset_index(drop=True)

train_df.to_csv('train.csv', index=False)
test_df.to_csv('test.csv', index=False)


In [ ]:
df1 = pd.read_csv('train.csv')
df1.sample(5)

Train

In [ ]:
import argparse
import bitsandbytes as bnb
from datasets import load_dataset
from functools import partial
import os
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, AutoPeftModelForCausalLM
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, set_seed, Trainer, TrainingArguments, BitsAndBytesConfig, \
    DataCollatorForLanguageModeling, Trainer, TrainingArguments
from datasets import Dataset
import pandas as pd
import argparse
from huggingface_hub import login


seed = 42
set_seed(seed)

#TODO:
# !huggingface-cli login
login(token = "")


def load_model(model_name, bnb_config):
    n_gpus = torch.cuda.device_count()
    max_memory = f'{40960}MB'

    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        quantization_config=bnb_config,
        device_map="auto", # dispatch efficiently the model on the available ressources
        max_memory = {i: max_memory for i in range(n_gpus)},
    )
    tokenizer = AutoTokenizer.from_pretrained(model_name, use_auth_token=True)

    tokenizer.pad_token = tokenizer.eos_token

    return model, tokenizer

def preprocess_batch(batch, tokenizer):
    """
    Tokenizing a batch
    """
    return tokenizer(
        batch["Prompt"],
        truncation=True,
    )


def create_bnb_config():
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16,
    )

    return bnb_config

def create_peft_config(modules):
    """
    Create Parameter-Efficient Fine-Tuning config for your model
    :param modules: Names of the modules to apply Lora to
    """
    config = LoraConfig(
        r=16,  # dimension of the updated matrices
        lora_alpha=64,  # parameter for scaling
        target_modules=modules,
        lora_dropout=0.1,  # dropout probability for layers
        bias="none",
        task_type="CAUSAL_LM",
    )

    return config


def find_all_linear_names(model):
    cls = bnb.nn.Linear4bit #if args.bits == 4 else (bnb.nn.Linear8bitLt if args.bits == 8 else torch.nn.Linear)
    lora_module_names = set()
    for name, module in model.named_modules():
        if isinstance(module, cls):
            names = name.split('.')
            lora_module_names.add(names[0] if len(names) == 1 else names[-1])

    if 'lm_head' in lora_module_names:  # needed for 16-bit
        lora_module_names.remove('lm_head')
    return list(lora_module_names)


def print_trainable_parameters(model, use_4bit=False):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        num_params = param.numel()
        # if using DS Zero 3 and the weights are initialized empty
        if num_params == 0 and hasattr(param, "ds_numel"):
            num_params = param.ds_numel

        all_param += num_params
        if param.requires_grad:
            trainable_params += num_params
    if use_4bit:
        trainable_params /= 2
    print(
        f"all params: {all_param:,d} || trainable params: {trainable_params:,d} || trainable%: {100 * trainable_params / all_param}"
    )

def get_max_length(model):
    conf = model.config
    max_length = None
    for length_setting in ["n_positions", "max_position_embeddings", "seq_length"]:
        max_length = getattr(model.config, length_setting, None)
        if max_length:
            print(f"Found max lenth: {max_length}")
            break
    if not max_length:
        max_length = 1024
        print(f"Using default max length: {max_length}")
    return max_length

def train(model, tokenizer, dataset, output_dir):
    # Apply preprocessing to the model to prepare it by
    # 1 - Enabling gradient checkpointing to reduce memory usage during fine-tuning
    model.gradient_checkpointing_enable()

    # 2 - Using the prepare_model_for_kbit_training method from PEFT
    model = prepare_model_for_kbit_training(model)

    # Get lora module names
    modules = find_all_linear_names(model)

    # Create PEFT config for these modules and wrap the model to PEFT
    peft_config = create_peft_config(modules)
    model = get_peft_model(model, peft_config)

    # Print information about the percentage of trainable parameters
    print_trainable_parameters(model)

    # Training parameters
    trainer = Trainer(
        model=model,
        train_dataset=dataset,
        args=TrainingArguments(
            per_device_train_batch_size=1,
            gradient_accumulation_steps=4,
            warmup_steps=2,
            max_steps=15,
            learning_rate=2e-4,
            fp16=True,
            logging_steps=1,
            output_dir="outputs",
            optim="paged_adamw_8bit",
        ),
        data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False)
    )

    model.config.use_cache = False  # re-enable for inference to speed up predictions for similar inputs


    dtypes = {}
    for _, p in model.named_parameters():
        dtype = p.dtype
        if dtype not in dtypes: dtypes[dtype] = 0
        dtypes[dtype] += p.numel()
    total = 0
    for k, v in dtypes.items(): total+= v
    for k, v in dtypes.items():
        print(k, v, v/total)

    do_train = True

    # Launch training
    print("Training...")

    if do_train:
        train_result = trainer.train()
        metrics = train_result.metrics
        trainer.log_metrics("train", metrics)
        trainer.save_metrics("train", metrics)
        trainer.save_state()
        print(metrics)

    ###

    # Saving model
    print("Saving last checkpoint of the model...")
    os.makedirs(output_dir, exist_ok=True)
    trainer.model.save_pretrained(output_dir)

    # Free memory for merging weights
    del model
    del trainer
    torch.cuda.empty_cache()

# TODO 这里file_path改成自己的csv 确保你的数据在Prompt的column里
def main(file_path = "/content/train.csv", output_dir = "final_checkpoint"):
    #Models that need to be fine-tuned
    model_name = "meta-llama/Llama-2-7b-hf"
    bnb_config = create_bnb_config()
    model, tokenizer = load_model(model_name, bnb_config)
    df = pd.read_csv(file_path)
    dataset = Dataset.from_pandas(df)
    _preprocessing_function = partial(preprocess_batch, tokenizer=tokenizer)

    dataset = dataset.map(
            _preprocessing_function,
            batched=True,
        )
    dataset = dataset.shuffle(seed=seed)

    output_dir = output_dir
    train(model, tokenizer, dataset, output_dir)

main("/content/train.csv", "final_checkpoint")

In [ ]:
text = """### Instruction:
Is the first node connect with the last node?

###Input:
node1 has relation_1 with node2, node2 has relation_2 with node 3

###Response:

"""

# Specify device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)
# Tokenize input text
inputs = tokenizer(text, return_tensors="pt").to(device)

# Get answer
# (Adjust max_new_tokens variable as you wish (maximum number of tokens the model can generate to answer the input))
outputs = model.generate(input_ids=inputs["input_ids"].to(device), attention_mask=inputs["attention_mask"], max_new_tokens=50, pad_token_id=tokenizer.eos_token_id)

# Decode output & print it
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

In [ ]:
model = AutoPeftModelForCausalLM.from_pretrained("final_checkpoint", device_map="auto", torch_dtype=torch.bfloat16)
model = model.merge_and_unload()

output_merged_dir = "results/llama2/final_merged_checkpoint"
os.makedirs(output_merged_dir, exist_ok=True)
model.save_pretrained(output_merged_dir, safe_serialization=True)

# save tokenizer for easy inference
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-hf")
tokenizer.save_pretrained(output_merged_dir)

In [ ]:
!zip -r CoTlink.zip /content/results

**Link Prediction without ICL**

In [ ]:
import pandas as pd

# 加载CSV文件
csv_file = 'test.csv'  # 请替换为你的CSV文件名
df = pd.read_csv(csv_file)

# 在所有的'input_text'数据前面加上"###Input:"
df['input_text'] = "###Input: \n" + df['input_text']

# 保存更改后的DataFrame到新的CSV文件
df.to_csv('modified_' + csv_file, index=False)

print("Updated CSV file has been saved as 'modified_" + csv_file + "'.")

In [ ]:
# import pandas as pd

# # 加载CSV文件
# csv_file = 'modified_test.csv'  # 替换为你的CSV文件名
# df = pd.read_csv(csv_file)

# reference_text = "###Input: node_60922 not connected with node_14564. node_60922 not connected with node_14564. node_60922 not connected with node_14564. node_14564 has relation_131 with node_17958. node_17958 has relation_6 with node_17474. Is node 66972 connected with node 17474?"
# reference_length = len(reference_text)

# # 选择长度小于等于reference_text长度的行
# filtered_df = df[df['input_text'].str.len() <= reference_length]

# # 保存或处理过滤后的DataFrame
# filtered_df.to_csv('filtered_' + csv_file, index=False)  # 如果需要，保存到新的CSV文件
# num_rows = len(filtered_df)
# print(f"The new CSV file will have {num_rows} rows.")

# df1 = pd.read_csv('filtered_modified_test.csv')
# df1.sample(5)

In [ ]:
text = """###Input:
node_58627 has relation_86 with node_1290. node_1290 has relation_2 with node_18074. node_18074 has relation_2 with node_14699. node_14699 has relation_2 with node_18079. Is node 58627 connected with node 18079?"""

# Specify device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Tokenize input text
inputs = tokenizer(text, return_tensors="pt").to(device)

# Get answer
# (Adjust max_new_tokens variable as you wish (maximum number of tokens the model can generate to answer the input))
outputs = model.generate(input_ids=inputs["input_ids"].to(device), attention_mask=inputs["attention_mask"], max_new_tokens=130, pad_token_id=tokenizer.eos_token_id)

# Decode output & print it
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

In [ ]:
import pandas as pd
from sklearn.metrics import f1_score


y_true = []
y_pred = []

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# 读取CSV文件
csv_file = '/content/modified_test.csv'
df = pd.read_csv(csv_file)

# 限制测试数量
test_limit = 2000
# 计数器
accurate_count = 0

for index, row in df.iterrows():
    if index <= test_limit:  # 检查是否已达到测试数量的限制
          break
    # 获取输入文本
    input_text = row['input_text']

    # 使用模型生成回答
    inputs = tokenizer(input_text, return_tensors="pt").to(device)
    outputs = model.generate(input_ids=inputs["input_ids"].to(device), attention_mask=inputs["attention_mask"], max_new_tokens=150, pad_token_id=tokenizer.eos_token_id)
    model_answer = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # 检查模型回答和期望输出中是否包含'yes'或'Yes'
    model_has_yes = 'yes' in model_answer.lower()
    expected_has_yes = 'yes' in row['output_text'].lower()

    # print("model_has_yes: ", model_has_yes)
    # print("expected_has_yes: ", expected_has_yes)

    y_true.append(expected_has_yes)
    y_pred.append(model_has_yes)


    # 判断准确性
    if model_has_yes == expected_has_yes:
        accurate_count += 1
        print(accurate_count)

    print(index)

# 计算准确率
f1 = f1_score(y_true, y_pred, pos_label=True)
accuracy = accurate_count / min(len(df), test_limit)
print(f'Accuracy: {accuracy:.3f}')
print(f'F1 Score: {f1:.3f}')

In [ ]:
f1 = f1_score(y_true, y_pred, pos_label=True)
accuracy = accurate_count / min(len(df), test_limit)
print(f'Accuracy: {accuracy:.3f}')
print(f'F1 Score: {f1:.3f}')

Accuracy: 0.953
F1 Score: 0.975


**Link Prediction with ICL**

In [ ]:
df = pd.read_csv('test.csv')
count = len(df)

In [ ]:
c = 0

fieldnames = ['input_text', 'output_text']
with open("train.csv") as train:
  with open("context.csv", mode="w", newline='') as context:
      writer_context = csv.DictWriter(context, fieldnames=fieldnames)
      writer_context.writeheader()
      reader_train = csv.reader(train)
      # Iterate over each row in the CSV file
      for row in reader_train:
          input = row[1]
          output = row[2]
          if input == 'input_text' or output == 'output_text':
            continue

          input = input.split('.')
          input = input[:-1]
          input = '.'.join(input)

          output = output.split('.')
          words = output[-3:-1]
          # relationship = words[3]
          # print(words)
          words = '.'.join(words)

          if c < count and len(input) < 30:
            cont = '###Context: {}.\n\n###Input:'.format(input + '.' + words)
            writer_context.writerow({'input_text': cont, 'output_text': words})
            c += 1
            # print(c)



In [ ]:
import pandas as pd
import random

# 加载CSV文件
context_df = pd.read_csv('context.csv')
test_df = pd.read_csv('test.csv')

# 对test.csv中的每个input_text随机添加一个context.csv中的input_text
for index, row in test_df.iterrows():
    # 随机选择一个context
    random_context = random.choice(context_df['input_text'])
    # 将context和test中的文本合并，并直接覆盖原有的input_text
    test_df.at[index, 'input_text'] = random_context + " " + row['input_text']

# 保存更新后的test_df到CSV文件
test_df.to_csv('augmented_test.csv', index=False)

print("Augmented test CSV file has been created.")


In [ ]:
# df1 = pd.read_csv('context.csv')
df1 = pd.read_csv('augmented_test.csv')

df1.sample(5)

In [ ]:
import pandas as pd
from sklearn.metrics import f1_score


y_true = []
y_pred = []

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# 读取CSV文件
csv_file = '/content/augmented_test.csv'
df = pd.read_csv(csv_file)

# 限制测试数量
test_limit = 2000
# 计数器
accurate_count = 0

for index, row in df.iterrows():
    if index <= test_limit:  # 检查是否已达到测试数量的限制
          break
    # 获取输入文本
    input_text = row['input_text']

    # 使用模型生成回答
    inputs = tokenizer(input_text, return_tensors="pt").to(device)
    outputs = model.generate(input_ids=inputs["input_ids"].to(device), attention_mask=inputs["attention_mask"], max_new_tokens=150, pad_token_id=tokenizer.eos_token_id)
    model_answer = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # 检查模型回答和期望输出中是否包含'yes'或'Yes'
    model_has_yes = 'yes' in model_answer.lower()
    expected_has_yes = 'yes' in row['output_text'].lower()

    # print("model_has_yes: ", model_has_yes)
    # print("expected_has_yes: ", expected_has_yes)

    y_true.append(expected_has_yes)
    y_pred.append(model_has_yes)


    # 判断准确性
    if model_has_yes == expected_has_yes:
        accurate_count += 1
        print(accurate_count)

    print(index)

# 计算准确率
f1 = f1_score(y_true, y_pred, pos_label=True)
accuracy = accurate_count / min(len(df), test_limit)
print(f'Accuracy: {accuracy:.3f}')
print(f'F1 Score: {f1:.3f}')

In [ ]:
f1 = f1_score(y_true, y_pred, pos_label=True)
accuracy = accurate_count / min(len(df), test_limit)
print(f'Accuracy: {accuracy:.3f}')
print(f'F1 Score: {f1:.3f}')

# *Relation prediction without In-context learning*


In [ ]:
import csv
import pandas as pd
# context = 'Given node_2075 has relation_4 with node_12648. The relation between the first node and last node is _member_meronym. '
# instruction = 'Answer the following multiple-choice question by choosing one of these options: '
# option = ''
instruction = 'Answer the following question step-by-step. '
count = 0

question = 'The relationship between the first node and the last node is ?'
fieldnames = ['input_text', 'output_text']
with open("relation2id.txt", "r") as file:
    relations = int(file.readline())
    for line in file:
        relation, relation_id = line.strip().split("\t")
    #     instruction += relation + ', '
    # instruction = instruction[:-2] + '. '

with open("test.csv") as test:
    with open("relation.csv", mode="w", newline='') as icl:
      writer_icl = csv.DictWriter(icl, fieldnames=fieldnames)
      writer_icl.writeheader()
      reader = csv.reader(test)


      for row in reader:
          # print(row)
          input = row[1]
          output = row[2]
          if input == 'input_text' or output == 'output_text':
            continue
          input = input.split('.')
          input = input[:-1]
          input = '.'.join(input)
          # print(input)
          input = instruction + input + '. ' + question
          # input = input + '. ' + instruction + question

          output = output.split('.')
          words = output[-3].split()
          relationship = words[3]
          # print(relationship)
          output = output[:-2]
          output = '.'.join(output)
          # print(output)
          output += '. The relationship between the first node and the last node is {}.'.format(relationship)
          # print(output)
          if len(input) < 550:
            writer_icl.writerow({'input_text': input, 'output_text': output})
            count+=1



In [ ]:
# df1 = pd.read_csv('context.csv')
df1 = pd.read_csv('relation.csv')

df1.sample(5)

In [ ]:
import pandas as pd
from sklearn.metrics import f1_score, accuracy_score

csv_file = '/content/relation.csv'
df = pd.read_csv(csv_file)

# 初始化计数器和结果列表
tp = fp = fn = 0
y_true = []
y_pred = []
test_limit = 500

for index, row in df.iterrows():
    if index <= test_limit:  # 检查是否已达到测试数量的限制
        break
    input_text = row['input_text']
    expected_word = row['output_text'].rstrip('.').split()[-1].lower()

    # 使用模型生成回答
    inputs = tokenizer(input_text, return_tensors="pt").to(device)
    outputs = model.generate(input_ids=inputs["input_ids"].to(device), attention_mask=inputs["attention_mask"], max_new_tokens=50, pad_token_id=tokenizer.eos_token_id)
    model_answer = tokenizer.decode(outputs[0], skip_special_tokens=True).lower()

    # 检查模型回答中是否包含最后一个单词
    contains_word = expected_word in model_answer.rstrip('.').split()

    # 更新计数器
    if contains_word:
        tp += 1
        y_pred.append(1)
    else:
        fn += 1
        y_pred.append(0)
    y_true.append(1)

# 计算准确度和F1分数
accuracy = accuracy_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)

print(f'Accuracy: {accuracy:.3f}')
print(f'F1 Score: {f1:.3f}')


# *Relation prediction with In-context learning*




In [ ]:
import csv
import pandas as pd
# context = 'Given node_2075 has relation_4 with node_12648. The relation between the first node and last node is _member_meronym. '
# instruction = 'Answer the following multiple-choice question by choosing one of these options: '
# option = ''
instruction = 'Answer the following question step-by-step. '
count = 0

question = 'The relationship between the first node and the last node is ?'
fieldnames = ['input_text', 'output_text']
with open("relation2id.txt", "r") as file:
    relations = int(file.readline())
    for line in file:
        relation, relation_id = line.strip().split("\t")
    #     instruction += relation + ', '
    # instruction = instruction[:-2] + '. '

with open("test.csv") as test:
    with open("test_icl.csv", mode="w", newline='') as icl:
      writer_icl = csv.DictWriter(icl, fieldnames=fieldnames)
      writer_icl.writeheader()
      reader = csv.reader(test)


      for row in reader:
          # print(row)
          input = row[1]
          output = row[2]
          if input == 'input_text' or output == 'output_text':
            continue
          input = input.split('.')
          input = input[:-1]
          input = '.'.join(input)
          # print(input)
          input = instruction + input + '. ' + question
          # input = input + '. ' + instruction + question

          output = output.split('.')
          words = output[-3].split()
          relationship = words[3]
          # print(relationship)
          output = output[:-2]
          output = '.'.join(output)
          # print(output)
          output += '. The relationship between the first node and the last node is {}.'.format(relationship)
          # print(output)
          if len(input) < 550:
            writer_icl.writerow({'input_text': input, 'output_text': output})
            count+=1



In [ ]:
c = 0

with open("train.csv") as train:
  with open("context.csv", mode="w", newline='') as context:
      writer_context = csv.DictWriter(context, fieldnames=fieldnames)
      writer_context.writeheader()
      reader_train = csv.reader(train)
      # Iterate over each row in the CSV file
      for row in reader_train:
          input = row[1]
          output = row[2]
          if input == 'input_text' or output == 'output_text':
            continue

          input = input.split('.')
          input = input[:-1]
          input = '.'.join(input)

          output = output.split('.')
          words = output[-3].split()
          relationship = words[3]

          if c < count and len(input) < 50:
            cont = '###Context: {}. The relationship between the first node and last node is {}.\n\n###Input:'.format(input, relationship)
            writer_context.writerow({'input_text': cont, 'output_text': relationship})
            c += 1
            # print(c)



In [ ]:
import pandas as pd
import random

# 加载CSV文件
context_df = pd.read_csv('context.csv')
test_df = pd.read_csv('test_icl.csv')

# 对test.csv中的每个input_text随机添加一个context.csv中的input_text
for index, row in test_df.iterrows():
    # 随机选择一个context
    random_context = random.choice(context_df['input_text'])
    # 将context和test中的文本合并，并直接覆盖原有的input_text
    test_df.at[index, 'input_text'] = random_context + " " + row['input_text']

# 保存更新后的test_df到CSV文件
test_df.to_csv('icl_relation.csv', index=False)

print("Augmented test CSV file has been created.")


In [ ]:
# read csv files
df1 = pd.read_csv('context.csv')
df2 = pd.read_csv('test_icl.csv')

# combine dataframes by interleaving rows

combined_df = pd.concat([df1, df2]).sort_index(kind='merge')

# write to new csv file
combined_df.to_csv('icl_relation.csv', index=False)

In [ ]:
# df1 = pd.read_csv('context.csv')
df1 = pd.read_csv('icl_relation.csv')

df1.sample(5)

In [ ]:
import pandas as pd
from sklearn.metrics import f1_score, accuracy_score

csv_file = '/content/icl_relation.csv'
df = pd.read_csv(csv_file)

# 初始化计数器和结果列表
tp = fp = fn = 0
y_true = []
y_pred = []
test_limit = 500

for index, row in df.iterrows():
    if index <= test_limit:  # 检查是否已达到测试数量的限制
        break
    input_text = row['input_text']
    expected_word = row['output_text'].rstrip('.').split()[-1].lower()

    # 使用模型生成回答
    inputs = tokenizer(input_text, return_tensors="pt").to(device)
    outputs = model.generate(input_ids=inputs["input_ids"].to(device), attention_mask=inputs["attention_mask"], max_new_tokens=50, pad_token_id=tokenizer.eos_token_id)
    model_answer = tokenizer.decode(outputs[0], skip_special_tokens=True).lower()

    # 检查模型回答中是否包含最后一个单词
    contains_word = expected_word in model_answer.rstrip('.').split()

    # 更新计数器
    if contains_word:
        tp += 1
        y_pred.append(1)
    else:
        fn += 1
        y_pred.append(0)
    y_true.append(1)

# 计算准确度和F1分数
accuracy = accuracy_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)

print(f'Accuracy: {accuracy:.3f}')
print(f'F1 Score: {f1:.3f}')


# *KG-LLM(Standard)*

In [ ]:
input_file = open("train2id.txt", "r")
output_file = open("output.txt", "w")

# total number of lines
number = int(input_file.readline())

nodes = set()

graph = {}

for i in range(number):
    content = input_file.readline()
    node1, node2, relation = content.strip().split()

    nodes.add(node1)
    # nodes.add(node2)

    relation = int(relation)

    # Check if the first node already exists in the dictionary
    if node1 not in graph:
        # If not, create a new dictionary for the node
        graph[node1] = {}
    # Add the neighboring node and the relationship to the dictionary for node1
    graph[node1][node2] = relation


node_list = list(nodes)
node_list2 = list(nodes)

In [ ]:

# os.environ["CUDA_VISIBLE_DEVICES"] = "1"

# relation2id = {}

# with open("relation2id.txt", "r") as file:
#     relations = int(file.readline())
#     for line in file:
#         relation, relation_id = line.strip().split("\t")
#         relation2id[int(relation_id)] = relation

unique_rows = set()

# change size if you want, but larger the size, smaller the data
size = 30
# how many positive and negative data set you want to train, the more dataset, the more time to train
total = 60000
fieldnames = ['Prompt', 'input_text', 'output_text']

# instruction = 'Answer the following yes/no question by reasoning step-by-step. '

with open("train_data.csv", mode="w", newline='') as tra:
  with open("positive_data.csv", mode="w", newline='') as pos:
    with open("negative_data.csv",  mode="w", newline='') as neg:

        # Create a CSV writer object and write the headers to the file
        writer_pos = csv.DictWriter(pos, fieldnames=fieldnames)
        writer_pos.writeheader()

        writer_neg = csv.DictWriter(neg, fieldnames=fieldnames)
        writer_neg.writeheader()

        writer_tra = csv.DictWriter(tra, fieldnames=fieldnames)
        writer_tra.writeheader()


        def dfs(graph, size):
            pos_count = 0
            neg_count = 0
            times = 0
            term = True

            while times < total:
                visited = set()
                kg = []
                graph_size = random.randint(2, size)
                first_node = random.choice(node_list)
                visited.add(first_node)
                last_node = ""
                previous_node = first_node
                stack = [first_node]
                input_text = ""
                output_text = ""
                while len(visited) < graph_size:
                    if previous_node not in graph or set(graph[previous_node].keys()).issubset(visited):
                        node = random.choice(node_list)
                        while node in visited:
                            node = random.choice(node_list)
                        input_text += 'node_{} not connected with node_{}. '.format(previous_node, node)
                        # output_text += 'node_{} not connected with node_{} means there is no relationship between node_{} and node_{}. '.format(previous_node, node, previous_node, node)
                        visited.add(node)
                        previous_node = node
                    else:
                        node = random.choice(list(graph[previous_node].keys()))
                        while node in visited:
                            node = random.choice(list(graph[previous_node].keys()))
                        relation = graph[previous_node][node]
                        # r = relation2id[relation]
                        input_text += 'node_{} has relation_{} with node_{}. '.format(previous_node, relation, node)
                        # output_text += 'node_{} has relation_{} with node_{}, means node_{} {} node_{}. '.format(previous_node, relation, node, previous_node, r, node)
                        visited.add(node)
                        previous_node = node
                    if len(visited) == graph_size:
                        last_node = previous_node

                # input_text += 'Answer the following yes/no question by reasoning step-by-step. Is the first node connnected with the last node?'
                was = len(unique_rows)
                unique_rows.add(input_text)
                if len(unique_rows) > was:
                  # if pos_count < int(total/2):
                    if first_node in graph and last_node in graph[first_node] and term:
                      # if pos_count < int(total/2):
                        # final_relation = relation2id[graph[first_node][last_node]]
                        output_text += 'The answer is yes.'
                        prompt = 'Is node {} connnected with node {}?'.format(first_node, last_node)
                        comb = "###Input:\n" + input_text + prompt + "\n\n###Response:\n" + output_text
                        writer_pos.writerow({'Prompt': comb, 'input_text': input_text + prompt, 'output_text': output_text})
                        writer_tra.writerow({'Prompt': comb, 'input_text': input_text + prompt, 'output_text': output_text})
                        pos_count += 1
                        term = False
                        times += 1
                    elif last_node in graph and first_node in graph[last_node] and term:
                      # if pos_count < int(total/2):
                        # final_relation = relation2id[graph[last_node][first_node]]
                        output_text += 'The answer is yes.'
                        prompt = 'Is node {} connected with node {}?'.format(last_node, first_node)
                        comb = "###Input:\n" + input_text + prompt + "\n\n###Response:\n" + output_text
                        writer_pos.writerow({'Prompt': comb, 'input_text': input_text + prompt, 'output_text': output_text})
                        writer_tra.writerow({'Prompt': comb, 'input_text': input_text + prompt, 'output_text': output_text})
                        pos_count += 1
                        term = False
                        times += 1
                  # elif neg_count < int(total//2):
                    elif not term:
                      # if neg_count < int(total/2):
                        output_text += 'The answer is no.'
                        prompt = 'Is node {} connected with node {}?'.format(first_node, last_node)
                        comb = "###Input:\n" + input_text + prompt + "\n\n###Response:\n" + output_text
                        writer_neg.writerow({'Prompt': comb, 'input_text': input_text + prompt, 'output_text': output_text})
                        writer_tra.writerow({'Prompt': comb, 'input_text': input_text + prompt, 'output_text': output_text})
                        neg_count += 1
                        term = True
                        times += 1
                else:
                  continue


            print(pos_count)
            print(neg_count)

        dfs(graph, size)

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# 读取CSV文件
positive_df = pd.read_csv('positive_data.csv')
negative_df = pd.read_csv('negative_data.csv')

# 各自分割数据集为80%的训练集和20%的测试集
train_pos, test_pos = train_test_split(positive_df, test_size=0.2, random_state=42)
train_neg, test_neg = train_test_split(negative_df, test_size=0.2, random_state=42)

# 合并训练集和测试集
train_df = pd.concat([train_pos, train_neg])
test_df = pd.concat([test_pos, test_neg])

# 打乱训练集和测试集
train_df = train_df.sample(frac=1, random_state=42).reset_index(drop=True)
test_df = test_df.sample(frac=1, random_state=42).reset_index(drop=True)

train_df.to_csv('train.csv', index=False)
test_df.to_csv('test.csv', index=False)


In [ ]:
import argparse
import bitsandbytes as bnb
from datasets import load_dataset
from functools import partial
import os
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, AutoPeftModelForCausalLM
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, set_seed, Trainer, TrainingArguments, BitsAndBytesConfig, \
    DataCollatorForLanguageModeling, Trainer, TrainingArguments
from datasets import Dataset
import pandas as pd
import argparse
from huggingface_hub import login


seed = 42
set_seed(seed)

#TODO:
# !huggingface-cli login
login(token = "")


def load_model(model_name, bnb_config):
    n_gpus = torch.cuda.device_count()
    max_memory = f'{40960}MB'

    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        quantization_config=bnb_config,
        device_map="auto", # dispatch efficiently the model on the available ressources
        max_memory = {i: max_memory for i in range(n_gpus)},
    )
    tokenizer = AutoTokenizer.from_pretrained(model_name, use_auth_token=True)

    tokenizer.pad_token = tokenizer.eos_token

    return model, tokenizer

def preprocess_batch(batch, tokenizer):
    """
    Tokenizing a batch
    """
    return tokenizer(
        batch["Prompt"],
        truncation=True,
    )


def create_bnb_config():
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16,
    )

    return bnb_config

def create_peft_config(modules):
    """
    Create Parameter-Efficient Fine-Tuning config for your model
    :param modules: Names of the modules to apply Lora to
    """
    config = LoraConfig(
        r=16,  # dimension of the updated matrices
        lora_alpha=64,  # parameter for scaling
        target_modules=modules,
        lora_dropout=0.1,  # dropout probability for layers
        bias="none",
        task_type="CAUSAL_LM",
    )

    return config


def find_all_linear_names(model):
    cls = bnb.nn.Linear4bit #if args.bits == 4 else (bnb.nn.Linear8bitLt if args.bits == 8 else torch.nn.Linear)
    lora_module_names = set()
    for name, module in model.named_modules():
        if isinstance(module, cls):
            names = name.split('.')
            lora_module_names.add(names[0] if len(names) == 1 else names[-1])

    if 'lm_head' in lora_module_names:  # needed for 16-bit
        lora_module_names.remove('lm_head')
    return list(lora_module_names)


def print_trainable_parameters(model, use_4bit=False):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        num_params = param.numel()
        # if using DS Zero 3 and the weights are initialized empty
        if num_params == 0 and hasattr(param, "ds_numel"):
            num_params = param.ds_numel

        all_param += num_params
        if param.requires_grad:
            trainable_params += num_params
    if use_4bit:
        trainable_params /= 2
    print(
        f"all params: {all_param:,d} || trainable params: {trainable_params:,d} || trainable%: {100 * trainable_params / all_param}"
    )

def get_max_length(model):
    conf = model.config
    max_length = None
    for length_setting in ["n_positions", "max_position_embeddings", "seq_length"]:
        max_length = getattr(model.config, length_setting, None)
        if max_length:
            print(f"Found max lenth: {max_length}")
            break
    if not max_length:
        max_length = 1024
        print(f"Using default max length: {max_length}")
    return max_length

def train(model, tokenizer, dataset, output_dir):
    # Apply preprocessing to the model to prepare it by
    # 1 - Enabling gradient checkpointing to reduce memory usage during fine-tuning
    model.gradient_checkpointing_enable()

    # 2 - Using the prepare_model_for_kbit_training method from PEFT
    model = prepare_model_for_kbit_training(model)

    # Get lora module names
    modules = find_all_linear_names(model)

    # Create PEFT config for these modules and wrap the model to PEFT
    peft_config = create_peft_config(modules)
    model = get_peft_model(model, peft_config)

    # Print information about the percentage of trainable parameters
    print_trainable_parameters(model)

    # Training parameters
    trainer = Trainer(
        model=model,
        train_dataset=dataset,
        args=TrainingArguments(
            per_device_train_batch_size=1,
            gradient_accumulation_steps=4,
            warmup_steps=2,
            max_steps=15,
            learning_rate=2e-4,
            fp16=True,
            logging_steps=1,
            output_dir="outputs",
            optim="paged_adamw_8bit",
        ),
        data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False)
    )

    model.config.use_cache = False  # re-enable for inference to speed up predictions for similar inputs


    dtypes = {}
    for _, p in model.named_parameters():
        dtype = p.dtype
        if dtype not in dtypes: dtypes[dtype] = 0
        dtypes[dtype] += p.numel()
    total = 0
    for k, v in dtypes.items(): total+= v
    for k, v in dtypes.items():
        print(k, v, v/total)

    do_train = True

    # Launch training
    print("Training...")

    if do_train:
        train_result = trainer.train()
        metrics = train_result.metrics
        trainer.log_metrics("train", metrics)
        trainer.save_metrics("train", metrics)
        trainer.save_state()
        print(metrics)

    ###

    # Saving model
    print("Saving last checkpoint of the model...")
    os.makedirs(output_dir, exist_ok=True)
    trainer.model.save_pretrained(output_dir)

    # Free memory for merging weights
    del model
    del trainer
    torch.cuda.empty_cache()

# TODO 这里file_path改成自己的csv 确保你的数据在Prompt的column里
def main(file_path = "/content/train.csv", output_dir = "final_checkpoint"):
    #Models that need to be fine-tuned
    model_name = "meta-llama/Llama-2-7b-hf"
    bnb_config = create_bnb_config()
    model, tokenizer = load_model(model_name, bnb_config)
    df = pd.read_csv(file_path)
    dataset = Dataset.from_pandas(df)
    _preprocessing_function = partial(preprocess_batch, tokenizer=tokenizer)

    dataset = dataset.map(
            _preprocessing_function,
            batched=True,
        )
    dataset = dataset.shuffle(seed=seed)

    output_dir = output_dir
    train(model, tokenizer, dataset, output_dir)

main("/content/train.csv", "final_checkpoint")

In [ ]:
model = AutoPeftModelForCausalLM.from_pretrained("final_checkpoint", device_map="auto", torch_dtype=torch.bfloat16)
model = model.merge_and_unload()

output_merged_dir = "results/llama2/final_merged_checkpoint"
os.makedirs(output_merged_dir, exist_ok=True)
model.save_pretrained(output_merged_dir, safe_serialization=True)

# save tokenizer for easy inference
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-hf")
tokenizer.save_pretrained(output_merged_dir)

Link prediction without ICL

In [ ]:
import pandas as pd

# 加载CSV文件
csv_file = 'test.csv'  # 请替换为你的CSV文件名
df = pd.read_csv(csv_file)

# 在所有的'input_text'数据前面加上"###Input:"
df['input_text'] = "###Input: \n" + df['input_text']

# 保存更改后的DataFrame到新的CSV文件
df.to_csv('modified_' + csv_file, index=False)

print("Updated CSV file has been saved as 'modified_" + csv_file + "'.")

In [ ]:
import pandas as pd
from sklearn.metrics import f1_score


y_true = []
y_pred = []

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# 读取CSV文件
csv_file = '/content/modified_test.csv'
df = pd.read_csv(csv_file)

# 限制测试数量
test_limit = 2000
# 计数器
accurate_count = 0

for index, row in df.iterrows():
    if index <= test_limit:  # 检查是否已达到测试数量的限制
          break
    # 获取输入文本
    input_text = row['input_text']

    # 使用模型生成回答
    inputs = tokenizer(input_text, return_tensors="pt").to(device)
    outputs = model.generate(input_ids=inputs["input_ids"].to(device), attention_mask=inputs["attention_mask"], max_new_tokens=150, pad_token_id=tokenizer.eos_token_id)
    model_answer = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # 检查模型回答和期望输出中是否包含'yes'或'Yes'
    model_has_yes = 'yes' in model_answer.lower()
    expected_has_yes = 'yes' in row['output_text'].lower()

    # print("model_has_yes: ", model_has_yes)
    # print("expected_has_yes: ", expected_has_yes)

    y_true.append(expected_has_yes)
    y_pred.append(model_has_yes)


    # 判断准确性
    if model_has_yes == expected_has_yes:
        accurate_count += 1
        print(accurate_count)

    print(index)

# 计算准确率
f1 = f1_score(y_true, y_pred, pos_label=True)
accuracy = accurate_count / min(len(df), test_limit)
print(f'Accuracy: {accuracy:.3f}')
print(f'F1 Score: {f1:.3f}')

Link prediction with ICL

In [ ]:
df = pd.read_csv('test.csv')
count = len(df)

In [ ]:
c = 0

fieldnames = ['input_text', 'output_text']
with open("train.csv") as train:
  with open("context.csv", mode="w", newline='') as context:
      writer_context = csv.DictWriter(context, fieldnames=fieldnames)
      writer_context.writeheader()
      reader_train = csv.reader(train)
      # Iterate over each row in the CSV file
      for row in reader_train:
          input = row[1]
          output = row[2]
          if input == 'input_text' or output == 'output_text':
            continue

          input = input.split('.')
          input = input[:-1]
          input = '.'.join(input)

          output = output.split('.')
          words = output[-3:-1]
          # relationship = words[3]
          # print(words)
          words = '.'.join(words)

          if c < count and len(input) < 30:
            cont = '###Context: {}.\n\n###Input:'.format(input + '.' + words)
            writer_context.writerow({'input_text': cont, 'output_text': words})
            c += 1
            # print(c)



In [ ]:
import pandas as pd
import random

# 加载CSV文件
context_df = pd.read_csv('context.csv')
test_df = pd.read_csv('test.csv')

# 对test.csv中的每个input_text随机添加一个context.csv中的input_text
for index, row in test_df.iterrows():
    # 随机选择一个context
    random_context = random.choice(context_df['input_text'])
    # 将context和test中的文本合并，并直接覆盖原有的input_text
    test_df.at[index, 'input_text'] = random_context + " " + row['input_text']

# 保存更新后的test_df到CSV文件
test_df.to_csv('augmented_test.csv', index=False)

print("Augmented test CSV file has been created.")


In [ ]:
# df1 = pd.read_csv('context.csv')
df1 = pd.read_csv('augmented_test.csv')

df1.sample(5)

In [ ]:
import pandas as pd
from sklearn.metrics import f1_score


y_true = []
y_pred = []

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# 读取CSV文件
csv_file = '/content/augmented_test.csv'
df = pd.read_csv(csv_file)

# 限制测试数量
test_limit = 2000
# 计数器
accurate_count = 0

for index, row in df.iterrows():
    if index <= test_limit:  # 检查是否已达到测试数量的限制
          break
    # 获取输入文本
    input_text = row['input_text']

    # 使用模型生成回答
    inputs = tokenizer(input_text, return_tensors="pt").to(device)
    outputs = model.generate(input_ids=inputs["input_ids"].to(device), attention_mask=inputs["attention_mask"], max_new_tokens=150, pad_token_id=tokenizer.eos_token_id)
    model_answer = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # 检查模型回答和期望输出中是否包含'yes'或'Yes'
    model_has_yes = 'yes' in model_answer.lower()
    expected_has_yes = 'yes' in row['output_text'].lower()

    # print("model_has_yes: ", model_has_yes)
    # print("expected_has_yes: ", expected_has_yes)

    y_true.append(expected_has_yes)
    y_pred.append(model_has_yes)


    # 判断准确性
    if model_has_yes == expected_has_yes:
        accurate_count += 1
        print(accurate_count)

    print(index)

# 计算准确率
f1 = f1_score(y_true, y_pred, pos_label=True)
accuracy = accurate_count / min(len(df), test_limit)
print(f'Accuracy: {accuracy:.3f}')
print(f'F1 Score: {f1:.3f}')

Relation prediction without ICL

In [ ]:
import csv
import pandas as pd
# context = 'Given node_2075 has relation_4 with node_12648. The relation between the first node and last node is _member_meronym. '
# instruction = 'Answer the following multiple-choice question by choosing one of these options: '
# option = ''
instruction = 'Answer the following question step-by-step. '
count = 0

question = 'The relationship between the first node and the last node is ?'
fieldnames = ['input_text', 'output_text']
with open("relation2id.txt", "r") as file:
    relations = int(file.readline())
    for line in file:
        relation, relation_id = line.strip().split("\t")
    #     instruction += relation + ', '
    # instruction = instruction[:-2] + '. '

with open("test.csv") as test:
    with open("relation.csv", mode="w", newline='') as icl:
      writer_icl = csv.DictWriter(icl, fieldnames=fieldnames)
      writer_icl.writeheader()
      reader = csv.reader(test)


      for row in reader:
          # print(row)
          input = row[1]
          output = row[2]
          if input == 'input_text' or output == 'output_text':
            continue
          input = input.split('.')
          input = input[:-1]
          input = '.'.join(input)
          # print(input)
          input = instruction + input + '. ' + question
          # input = input + '. ' + instruction + question

          output = output.split('.')
          words = output[-3].split()
          relationship = words[3]
          # print(relationship)
          output = output[:-2]
          output = '.'.join(output)
          # print(output)
          output += '. The relationship between the first node and the last node is {}.'.format(relationship)
          # print(output)
          if len(input) < 550:
            writer_icl.writerow({'input_text': input, 'output_text': output})
            count+=1



In [ ]:
# df1 = pd.read_csv('context.csv')
df1 = pd.read_csv('relation.csv')

df1.sample(5)

In [ ]:
import pandas as pd
from sklearn.metrics import f1_score, accuracy_score

csv_file = '/content/relation.csv'
df = pd.read_csv(csv_file)

# 初始化计数器和结果列表
tp = fp = fn = 0
y_true = []
y_pred = []
test_limit = 500

for index, row in df.iterrows():
    if index <= test_limit:  # 检查是否已达到测试数量的限制
        break
    input_text = row['input_text']
    expected_word = row['output_text'].rstrip('.').split()[-1].lower()

    # 使用模型生成回答
    inputs = tokenizer(input_text, return_tensors="pt").to(device)
    outputs = model.generate(input_ids=inputs["input_ids"].to(device), attention_mask=inputs["attention_mask"], max_new_tokens=50, pad_token_id=tokenizer.eos_token_id)
    model_answer = tokenizer.decode(outputs[0], skip_special_tokens=True).lower()

    # 检查模型回答中是否包含最后一个单词
    contains_word = expected_word in model_answer.rstrip('.').split()

    # 更新计数器
    if contains_word:
        tp += 1
        y_pred.append(1)
    else:
        fn += 1
        y_pred.append(0)
    y_true.append(1)

# 计算准确度和F1分数
accuracy = accuracy_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)

print(f'Accuracy: {accuracy:.3f}')
print(f'F1 Score: {f1:.3f}')


Relation prediction with ICL

In [ ]:
import csv
import pandas as pd
# context = 'Given node_2075 has relation_4 with node_12648. The relation between the first node and last node is _member_meronym. '
# instruction = 'Answer the following multiple-choice question by choosing one of these options: '
# option = ''
instruction = 'Answer the following question step-by-step. '
count = 0

question = 'The relationship between the first node and the last node is ?'
fieldnames = ['input_text', 'output_text']
with open("relation2id.txt", "r") as file:
    relations = int(file.readline())
    for line in file:
        relation, relation_id = line.strip().split("\t")
    #     instruction += relation + ', '
    # instruction = instruction[:-2] + '. '

with open("test.csv") as test:
    with open("test_icl.csv", mode="w", newline='') as icl:
      writer_icl = csv.DictWriter(icl, fieldnames=fieldnames)
      writer_icl.writeheader()
      reader = csv.reader(test)


      for row in reader:
          # print(row)
          input = row[1]
          output = row[2]
          if input == 'input_text' or output == 'output_text':
            continue
          input = input.split('.')
          input = input[:-1]
          input = '.'.join(input)
          # print(input)
          input = instruction + input + '. ' + question
          # input = input + '. ' + instruction + question

          output = output.split('.')
          words = output[-3].split()
          relationship = words[3]
          # print(relationship)
          output = output[:-2]
          output = '.'.join(output)
          # print(output)
          output += '. The relationship between the first node and the last node is {}.'.format(relationship)
          # print(output)
          if len(input) < 550:
            writer_icl.writerow({'input_text': input, 'output_text': output})
            count+=1



In [ ]:
c = 0

with open("train.csv") as train:
  with open("context.csv", mode="w", newline='') as context:
      writer_context = csv.DictWriter(context, fieldnames=fieldnames)
      writer_context.writeheader()
      reader_train = csv.reader(train)
      # Iterate over each row in the CSV file
      for row in reader_train:
          input = row[1]
          output = row[2]
          if input == 'input_text' or output == 'output_text':
            continue

          input = input.split('.')
          input = input[:-1]
          input = '.'.join(input)

          output = output.split('.')
          words = output[-3].split()
          relationship = words[3]

          if c < count and len(input) < 50:
            cont = '###Context: {}. The relationship between the first node and last node is {}.\n\n###Input:'.format(input, relationship)
            writer_context.writerow({'input_text': cont, 'output_text': relationship})
            c += 1
            # print(c)



In [ ]:
import pandas as pd
import random

# 加载CSV文件
context_df = pd.read_csv('context.csv')
test_df = pd.read_csv('test_icl.csv')

# 对test.csv中的每个input_text随机添加一个context.csv中的input_text
for index, row in test_df.iterrows():
    # 随机选择一个context
    random_context = random.choice(context_df['input_text'])
    # 将context和test中的文本合并，并直接覆盖原有的input_text
    test_df.at[index, 'input_text'] = random_context + " " + row['input_text']

# 保存更新后的test_df到CSV文件
test_df.to_csv('icl_relation.csv', index=False)

print("Augmented test CSV file has been created.")


In [ ]:
# read csv files
df1 = pd.read_csv('context.csv')
df2 = pd.read_csv('test_icl.csv')

# combine dataframes by interleaving rows

combined_df = pd.concat([df1, df2]).sort_index(kind='merge')

# write to new csv file
combined_df.to_csv('icl_relation.csv', index=False)

In [ ]:
# df1 = pd.read_csv('context.csv')
df1 = pd.read_csv('icl_relation.csv')

df1.sample(5)

In [ ]:
import pandas as pd
from sklearn.metrics import f1_score, accuracy_score

csv_file = '/content/icl_relation.csv'
df = pd.read_csv(csv_file)

# 初始化计数器和结果列表
tp = fp = fn = 0
y_true = []
y_pred = []
test_limit = 500

for index, row in df.iterrows():
    if index <= test_limit:  # 检查是否已达到测试数量的限制
        break
    input_text = row['input_text']
    expected_word = row['output_text'].rstrip('.').split()[-1].lower()

    # 使用模型生成回答
    inputs = tokenizer(input_text, return_tensors="pt").to(device)
    outputs = model.generate(input_ids=inputs["input_ids"].to(device), attention_mask=inputs["attention_mask"], max_new_tokens=50, pad_token_id=tokenizer.eos_token_id)
    model_answer = tokenizer.decode(outputs[0], skip_special_tokens=True).lower()

    # 检查模型回答中是否包含最后一个单词
    contains_word = expected_word in model_answer.rstrip('.').split()

    # 更新计数器
    if contains_word:
        tp += 1
        y_pred.append(1)
    else:
        fn += 1
        y_pred.append(0)
    y_true.append(1)

# 计算准确度和F1分数
accuracy = accuracy_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)

print(f'Accuracy: {accuracy:.3f}')
print(f'F1 Score: {f1:.3f}')
